In [35]:
from skimage import io
import matplotlib.pylab as plt
import numpy as np
from skimage.draw import line_nd
from skimage.util import img_as_ubyte, img_as_float64
import glob
from ipywidgets import interact, Dropdown, IntSlider, Layout, Checkbox, Button, HTML
from IPython.display import display

filename = "./images/Shepp_logan.jpg"
test = io.imread(filename)
#test = img_as_float64(test)

### Algorytm Bresenhama

In [36]:
def bresenham(x0, y0, x1, y1):
    points = []
    
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        points.append((y0, x0))
        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy

    return points

In [37]:
# makes image bigger (so that circle will be inside)
def pad_image(image):
    original_size = image.shape
    image_center = (original_size[0] // 2, original_size[1] // 2)
    required_radius = int(max(image_center) * np.sqrt(2))

    padding = max(0, required_radius - max(image_center))

    if padding > 0:
        image = np.pad(image, ((padding, padding), (padding, padding)), mode='constant', constant_values=0)
    
    return image

In [38]:
def normalize_image(image):
    image = np.maximum(image, 0)
    max_val = np.quantile(image, 0.999)
    image = np.clip(image / max_val, 0, 1) if max_val > 0 else np.zeros_like(image)
    return image

# Transformata Radona

In [39]:
def radon_transform(image, detectors = 180, detectors_angular_span = 180, step = 1):
    image_center = (image.shape[0] // 2, image.shape[1] // 2)
    image_radius = max(image_center) * np.sqrt(2)

    emiter_angles = np.deg2rad(np.arange(0, 360, step))
    radon_image = np.zeros((len(emiter_angles), detectors), dtype=image.dtype)

    detectors_angle = np.deg2rad(detectors_angular_span)
    detectors_gap = detectors_angle / (detectors - 1)

    for i, angle in enumerate(emiter_angles):
        emiter_pos = (int(image_radius * np.cos(angle) + image_center[0]), 
                      int(image_radius * np.sin(angle) + image_center[1]))

        for j in range(detectors):
            detector_pos = (int(image_radius * (np.cos(angle + np.pi - detectors_angle / 2 + j * detectors_gap)) + image_center[0]),
                            int(image_radius * (np.sin(angle + np.pi - detectors_angle / 2 + j * detectors_gap)) + image_center[1]))

            line_pixels = bresenham(emiter_pos[0], emiter_pos[1], detector_pos[0], detector_pos[1])
            
            valid_pixels = [(y, x) for y, x in line_pixels if 0 <= y < image.shape[0] and 0 <= x < image.shape[1]]

            if valid_pixels:
                radon_image[i, j] = np.mean([image[y, x] for y, x in valid_pixels])

    return radon_image

# Odwrócona transformata Radona

In [ ]:
def inverse_radon_transform(image, image_size, detectors_angular_span = 180, step = 1):
    reconstructed_image = np.zeros((image_size[0], image_size[1]), dtype=image.dtype)
    
    # get scans/detectors from image
    scans = image.shape[0]
    detectors = image.shape[1]

    image_center = (image_size[0] // 2, image_size[1] // 2)
    image_radius = max(image_center) * np.sqrt(2)

    emiter_angles = np.deg2rad(np.arange(0, 360, step))
    detectors_angle = np.deg2rad(detectors_angular_span)
    detectors_gap = detectors_angle / (detectors - 1)

    for i, angle in enumerate(emiter_angles):
        emiter_pos = (int(image_radius * np.cos(angle) + image_center[0]), 
                        int(image_radius * np.sin(angle) + image_center[1]))

        for j in range(detectors):
            detector_pos = (int(image_radius * (np.cos(angle + np.pi - detectors_angle / 2 + j * detectors_gap)) + image_center[0]),
                            int(image_radius * (np.sin(angle + np.pi - detectors_angle / 2 + j * detectors_gap)) + image_center[1]))

            line_pixels = bresenham(emiter_pos[0], emiter_pos[1], detector_pos[0], detector_pos[1])

            valid_pixels = [(y, x) for y, x in line_pixels if 0 <= y < image_size[0] and 0 <= x < image_size[1]]
            if valid_pixels:
                rows, cols = zip(*valid_pixels)
                reconstructed_image[rows, cols] += image[i, j] / len(valid_pixels)

    return reconstructed_image

# Filtr sinogramu

In [41]:
def filter_sinogram(image, kernel_size=21):
    # make it odd
    if kernel_size % 2 == 0:
        kernel_size += 1

    kernel = np.zeros(kernel_size)
    kernel_center = kernel_size // 2
    kernel[kernel_center] = 1
    for i in range(kernel_center + 1, kernel_size, 2):
        val = (-4 / np.pi**2) / (i - kernel_center)**2
        kernel[i] = val
        kernel[-(i + 1)] = val

    filtered_image = np.zeros((image.shape[0], image.shape[1]), dtype=image.dtype)
    for i in range(image.shape[0]):
        filtered_image[i, :] = np.convolve(image[i, :], kernel, mode='same')
    return filtered_image

# Interaktywne tworzenie tomografu

In [42]:
# global variables
selected_image_path = ""
detectors = 180
detectors_angular_span = 180
step = 1

is_filter = True

def set_parameters(image, d, arch, i):
    global selected_image_path, scans, detectors, detectors_angular_span, step
    selected_image_path = image
    detectors = d
    detectors_angular_span = arch
    step = i

def set_is_filter(val):
    global is_filter
    is_filter = val

In [43]:
def show_plot(arg):
    image = io.imread(selected_image_path)

    if image.ndim == 3:
        image = image[..., 0]
    
#    image = pad_image(image)
    
    sinogram = radon_transform(image, detectors=detectors, detectors_angular_span=detectors_angular_span, step=step)

    filtered_sinogram = []
    if is_filter:
        filtered_sinogram = filter_sinogram(img_as_float64(sinogram))
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(sinogram, cmap='gray', aspect='auto')
    axes[0].set_title("Sinogram (no filter)")
    axes[0].axis("off")

    axes[1].imshow(filtered_sinogram, cmap='gray', aspect='auto')
    axes[1].set_title("Sinogram (filter)")
    axes[1].axis("off")

    plt.show()
    
    inversed_sinogram = inverse_radon_transform(sinogram, image.shape, detectors_angular_span=detectors_angular_span, step=step)
    inversed_sinogram_filtered = inverse_radon_transform(filtered_sinogram, image.shape, detectors_angular_span=detectors_angular_span, step=step)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(inversed_sinogram, cmap='gray')
    axes[0].set_title("Inversed sinogram (no filter)")
    axes[0].axis("off")

    axes[1].imshow(inversed_sinogram_filtered, cmap='gray')
    axes[1].set_title("Inversed sinogram (filter)")
    axes[1].axis("off")

    plt.show()

# Interaktywny wybór parametrów i zdjęcia

In [44]:
filenames = glob.glob(".\\images\\*.jpg")

interact(set_parameters,
         image=Dropdown(options=filenames, value='.\\images\\Kropka.jpg', description='Filename', layout=Layout(width='80%'), style={'description_width': '200px'}),
         d=IntSlider(value=180, min=90, max=720, step=30, description='Amount of detectors:', layout=Layout(width='80%'), style={'description_width': '200px'}),
         arch=IntSlider(value=180, min=90, max=360, step=10, description='Detectors angular span:', layout=Layout(width='80%'), style={'description_width': '200px'}),
         i=IntSlider(value=2, min=1, max=10, step=1, description='Emiter-detectors rotate steps:', layout=Layout(width='80%'), style={'description_width': '200px'})
)

interact(set_is_filter, val=Checkbox(value=True, description='Sinogram filtering'))

button = Button(description="Show results")
button.on_click(show_plot)

display(button)

interactive(children=(Dropdown(description='Filename', index=3, layout=Layout(width='80%'), options=('.\\image…

interactive(children=(Checkbox(value=True, description='Sinogram filtering'), Output()), _dom_classes=('widget…

Button(description='Show results', style=ButtonStyle())